In [45]:
import pandas as pd
from datetime import datetime, timedelta


In [64]:
for i in range(2+1):
    print (i)

0
1
2


In [57]:
df = pd.read_csv('routes.csv')
df.head()

,source,destination,departure,arrival,flight_number,price,bags_allowed,bag_price
0,USM,HKT,2019-05-11T06:25:00,2019-05-11T07:25:00,PV404,24,1,9
1,USM,HKT,2019-05-12T12:15:00,2019-05-12T13:15:00,PV755,23,2,9
2,USM,HKT,2019-05-12T21:15:00,2019-05-12T22:15:00,PV729,25,1,14
3,USM,HKT,2019-05-11T14:50:00,2019-05-11T15:50:00,PV966,21,1,17
4,USM,HKT,2019-05-12T00:35:00,2019-05-12T01:35:00,PV398,24,1,14


In [55]:
df['departure'] =  pd.to_datetime(df['departure'], format='%Y-%m-%dT%H:%M:%S')
df['arrival'] =  pd.to_datetime(df['arrival'], format='%Y-%m-%dT%H:%M:%S')

flights = df[df.bags_allowed == 1].values.tolist()
flights = df.values.tolist()

flights = [
    {
        'flights': flight ,
        'route':[flight[0], flight[1]], 
        'schedule': flight[2:4], 
        'flight_numbers':[flight[4]], 
        'total_price': flight[5], 
        'max_bags_allowed': flight[6], 
        'total_bag_price': flight[7], 
        'can_be_extended': True
    } for flight in flights
]

for flight in (flight for flight in flights if flight['can_be_extended']):
    for next_flight in flights:
        if (next_flight['route'][0] == flight['route'][-1] and (timedelta(hours=1) <= next_flight['schedule'][0] - flight['schedule'][-1] <= timedelta(hours=4))):
            new_route =  {
                'flights': [flight['flights']] + [next_flight['flights']],
                'route':flight['route'] + [next_flight['route'][1]], 
                'schedule': flight['schedule'] + next_flight['schedule'], 
                'flight_numbers':flight['flight_numbers'] + next_flight['flight_numbers'], 
                'total_price': flight['total_price'] + next_flight['total_price'], 
                'max_bags_allowed': min(flight['max_bags_allowed'], next_flight['max_bags_allowed']),
                'total_bag_price': flight['total_bag_price'] + next_flight['total_bag_price'], 
                'can_be_extended': True
            }
            
            if new_route['route'][0] == new_route['route'][-1]:
                new_route['can_be_extended'] = False
            
            flights += [new_route]
    flight['can_be_extended'] == False

In [67]:
import sys
import json
from datetime import datetime, timedelta

import pandas as pd 

flights = df.values.tolist()

flights = [
    {
        'flights': [flight],
        'route':[flight[0], flight[1]], 
        'schedule': flight[2:4], 
        'flight_numbers':[flight[4]], 
        'total_price': flight[5], 
        'max_bags_allowed': flight[6], 
        'total_per_bag_price': flight[7], 
        'can_be_extended': True
    } for flight in flights
]

for flight in (flight for flight in flights if flight['can_be_extended']):
    for next_flight in flights:
        if (next_flight['route'][0] == flight['route'][-1] and (timedelta(hours=1) <= (datetime.strptime(next_flight['schedule'][0], '%Y-%m-%dT%H:%M:%S') - datetime.strptime(flight['schedule'][-1], '%Y-%m-%dT%H:%M:%S'))<= timedelta(hours=4))):
            new_route =  {
                'flights': [flight['flights']] + [next_flight['flights']],
                'route':flight['route'] + [next_flight['route'][1]], 
                'schedule': flight['schedule'] + next_flight['schedule'], 
                'flight_numbers':flight['flight_numbers'] + next_flight['flight_numbers'], 
                'total_price': flight['total_price'] + next_flight['total_price'], 
                'max_bags_allowed': min(flight['max_bags_allowed'], next_flight['max_bags_allowed']),
                'total_per_bag_price': flight['total_per_bag_price'] + next_flight['total_per_bag_price'], 
                'can_be_extended': True,
            }

            if new_route['route'][0] == new_route['route'][-1]:
                new_route['can_be_extended'] = False
            
            flights += [new_route]
    flight['can_be_extended'] == False

output = []

for flight in flights:
    if 'final_price' not in flight:
        for i in range (flight['max_bags_allowed'] + 1):
            print (i)
            flight['bags_selected'] = i
            flight['final_price'] = flight['total_price'] + flight['total_per_bag_price'] * i
            output.append(flight)
            print (flight)
        break

print (output)

0
{'flights': [['USM', 'HKT', '2019-05-11T06:25:00', '2019-05-11T07:25:00', 'PV404', 24, 1, 9]], 'route': ['USM', 'HKT'], 'schedule': ['2019-05-11T06:25:00', '2019-05-11T07:25:00'], 'flight_numbers': ['PV404'], 'total_price': 24, 'max_bags_allowed': 1, 'total_per_bag_price': 9, 'can_be_extended': True, 'bags_selected': 0, 'final_price': 24}
1
{'flights': [['USM', 'HKT', '2019-05-11T06:25:00', '2019-05-11T07:25:00', 'PV404', 24, 1, 9]], 'route': ['USM', 'HKT'], 'schedule': ['2019-05-11T06:25:00', '2019-05-11T07:25:00'], 'flight_numbers': ['PV404'], 'total_price': 24, 'max_bags_allowed': 1, 'total_per_bag_price': 9, 'can_be_extended': True, 'bags_selected': 1, 'final_price': 33}
[{'flights': [['USM', 'HKT', '2019-05-11T06:25:00', '2019-05-11T07:25:00', 'PV404', 24, 1, 9]], 'route': ['USM', 'HKT'], 'schedule': ['2019-05-11T06:25:00', '2019-05-11T07:25:00'], 'flight_numbers': ['PV404'], 'total_price': 24, 'max_bags_allowed': 1, 'total_per_bag_price': 9, 'can_be_extended': True, 'bags_selec

In [61]:
[[flight['route'], flight['max_bags_allowed']] for flight in flights]

[[['USM', 'HKT'], 1],
 [['USM', 'HKT'], 2],
 [['USM', 'HKT'], 1],
 [['USM', 'HKT'], 1],
 [['USM', 'HKT'], 1],
 [['USM', 'HKT'], 1],
 [['USM', 'HKT'], 1],
 [['USM', 'HKT'], 2],
 [['USM', 'HKT'], 2],
 [['USM', 'HKT'], 2],
 [['USM', 'HKT'], 2],
 [['HKT', 'USM'], 1],
 [['HKT', 'USM'], 1],
 [['HKT', 'USM'], 2],
 [['HKT', 'USM'], 1],
 [['HKT', 'DPS'], 1],
 [['HKT', 'USM'], 2],
 [['HKT', 'DPS'], 1],
 [['HKT', 'USM'], 2],
 [['HKT', 'USM'], 1],
 [['HKT', 'USM'], 1],
 [['BWN', 'DPS'], 1],
 [['BWN', 'DPS'], 1],
 [['BWN', 'DPS'], 1],
 [['BWN', 'DPS'], 1],
 [['BWN', 'DPS'], 1],
 [['BWN', 'DPS'], 2],
 [['BWN', 'DPS'], 2],
 [['BWN', 'DPS'], 1],
 [['BWN', 'DPS'], 1],
 [['BWN', 'DPS'], 2],
 [['BWN', 'DPS'], 2],
 [['DPS', 'HKT'], 1],
 [['DPS', 'BWN'], 2],
 [['DPS', 'BWN'], 1],
 [['DPS', 'HKT'], 1],
 [['DPS', 'HKT'], 2],
 [['DPS', 'HKT'], 1],
 [['DPS', 'HKT'], 2],
 [['DPS', 'HKT'], 1],
 [['DPS', 'BWN'], 1],
 [['DPS', 'BWN'], 1],
 [['USM', 'HKT', 'USM'], 1],
 [['USM', 'HKT', 'USM'], 1],
 [['USM', 'HKT', '

In [51]:
[[flight['route'], flight['max_bags_allowed']] for flight in flights]

[[['USM', 'HKT'], 1],
 [['USM', 'HKT'], 2],
 [['USM', 'HKT'], 1],
 [['USM', 'HKT'], 1],
 [['USM', 'HKT'], 1],
 [['USM', 'HKT'], 1],
 [['USM', 'HKT'], 1],
 [['USM', 'HKT'], 2],
 [['USM', 'HKT'], 2],
 [['USM', 'HKT'], 2],
 [['USM', 'HKT'], 2],
 [['HKT', 'USM'], 1],
 [['HKT', 'USM'], 1],
 [['HKT', 'USM'], 2],
 [['HKT', 'USM'], 1],
 [['HKT', 'DPS'], 1],
 [['HKT', 'USM'], 2],
 [['HKT', 'DPS'], 1],
 [['HKT', 'USM'], 2],
 [['HKT', 'USM'], 1],
 [['HKT', 'USM'], 1],
 [['BWN', 'DPS'], 1],
 [['BWN', 'DPS'], 1],
 [['BWN', 'DPS'], 1],
 [['BWN', 'DPS'], 1],
 [['BWN', 'DPS'], 1],
 [['BWN', 'DPS'], 2],
 [['BWN', 'DPS'], 2],
 [['BWN', 'DPS'], 1],
 [['BWN', 'DPS'], 1],
 [['BWN', 'DPS'], 2],
 [['BWN', 'DPS'], 2],
 [['DPS', 'HKT'], 1],
 [['DPS', 'BWN'], 2],
 [['DPS', 'BWN'], 1],
 [['DPS', 'HKT'], 1],
 [['DPS', 'HKT'], 2],
 [['DPS', 'HKT'], 1],
 [['DPS', 'HKT'], 2],
 [['DPS', 'HKT'], 1],
 [['DPS', 'BWN'], 1],
 [['DPS', 'BWN'], 1],
 [['USM', 'HKT', 'USM'], 1],
 [['USM', 'HKT', 'USM'], 1],
 [['USM', 'HKT', '